# Lecture 11- Langgraph Day 4

Simple tool call with Langgraph must for stable tasks

### Step 1: Setup LLM code

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file

# Corrected way to retrieve the API key
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    raise ValueError("GOOGLE_API_KEY not found. Make sure it's set in the .env file.")

os.environ["GOOGLE_API_KEY"] = api_key

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

In [2]:
# Step 1: Define a tool

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from tavily import TavilyClient

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000000))
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))


In [3]:
query = "Indian Food"

In [5]:
from langchain_core.messages import AIMessage
from langchain_core.tools import tool

from langgraph.prebuilt import ToolNode

In [6]:
@tool
def get_wikipedia_result(query: str):
    "Call to get wikipedia result"
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000000))
    return wikipedia.run({"query": query})

@tool
def get_tavily_results(query: str):
    "Call to get tavily results"
    response = tavily_client.search(query, max_results=10)
    content = ""
    for result in response['results']:
        content += result['content']
    return content

In [7]:
tool1 = [get_wikipedia_result]
tool2 = [get_tavily_results]
tool_node1 = ToolNode(tool1)
tool_node2 = ToolNode(tool2)

In [9]:
message_with_single_tool_call_wiki = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "get_wikipedia_result",
            "args": {"query": query},
            "id": "tool_call_1",
            "type": "tool_call",
        }
    ])

response_tool_1 = tool_node1.invoke({"messages": [message_with_single_tool_call_wiki]})

In [10]:
for m in response_tool_1['messages']:
    m.pretty_print()

================================= Tool Message =================================
Name: get_wikipedia_result

Page: Indian cuisine
Summary: Indian cuisine consists of a variety of regional and traditional cuisines native to the Indian subcontinent. Given the diversity in soil, climate, culture, ethnic groups, and occupations, these cuisines vary substantially and use locally available spices, herbs, vegetables, and fruits.
Indian food is also heavily influenced by religion, in particular Hinduism and Islam, cultural choices and traditions. Historical events such as invasions, trade relations, and colonialism have played a role in introducing certain foods to India. The Columbian discovery of the New World brought a number of new vegetables and fruits. A number of these such as potatoes, tomatoes, chillies, peanuts, and guava have become staples in many regions of India.
Indian cuisine has shaped the history of international relations; the spice trade between India and Europe was the pri

In [11]:
message_with_single_tool_call_tavily = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "get_tavily_results",
            "args": {"query": query},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ])

response_tool_2 = tool_node2.invoke({"messages": [message_with_single_tool_call_tavily]})

In [12]:
for m in response_tool_2['messages']:
    m.pretty_print()

================================= Tool Message =================================
Name: get_tavily_results

Indian cuisine consists of a diverse range of curries, rice dishes, meats, vegetables and breads, all flavored with a traditional range of spices. Whilst delicacies vary by region and state, there are many similarities in terms of spices and flavors, regardless of the geographical region.Indian food is known for its bold use of aromatic spices such as cumin, coriander, and turmeric, which create the complex and vibrant flavors that define the cuisine. Along with its well-known meat dishes, Indian cuisine also offers a variety of vegan options, including vegetable curries and lentil-based meals.India Food: 25 Most Popular Traditional Dishes To Eat In India - Rainforest Cruises Destinations Experiences Deals About Search Contact +1.888.215.3555 Agents Back Expedition Cruises South America Asia Bucket List Back Adventure Luxury Family Honeymoon Charter Solo Back Back Back Amazon Rive

In [13]:
system_message_summarize = f'''You are expert at a giving a detailed summary of the content given to you.
User Query: {query}

Wikipedia Result: {response_tool_1['messages']}

Tavily Result: {response_tool_2['messages']}

Always attibute the responses to sources (Wikipedia and Tavily)
'''

In [14]:
final_summary = llm.invoke(system_message_summarize)

In [15]:
print(final_summary.content)

Indian cuisine is incredibly diverse, varying substantially across regions due to differences in soil, climate, culture, ethnic groups, and occupations (Wikipedia).  This diversity is reflected in the use of locally available spices, herbs, vegetables, and fruits (Wikipedia).  Religious influences, particularly Hinduism and Islam, along with historical events like invasions, trade, and colonialism, have also shaped Indian food (Wikipedia).  The Columbian Exchange introduced staples like potatoes, tomatoes, chilies, peanuts, and guava (Wikipedia).

The cuisine is characterized by a wide range of curries, rice dishes, meats, vegetables, and breads, all flavored with a traditional array of spices (Tavily).  Common spices include cumin, coriander, and turmeric, creating complex and vibrant flavors (Tavily).  While regional variations exist, many similarities in spices and flavors remain across geographical areas (Tavily).  Northern India features rich curries and breads, while Southern Ind

In [16]:
with open("results.txt", "w") as file:
    file.write(final_summary.content)